# Ejemplo de R-Arcgis-Bridge con Datos Vector

Ejemplo del uso de r-arcgis-bridge con datos VECTOR. Se realiza la exploración del dataset SECOP I y un Feature Layer del Departamento Nacional de Planeación

![Use Kernel R](https://www.r-project.org/Rlogo.png) Use Kernel R

In [ ]:
require("RSocrata")
require("arcgisbinding")
require("stringi")
require("doBy")
require("sp")
require("sqldf")
require("spatialEco")
require("rmarkdown")
require("knitr")
require("dplyr")
require("reshape2")
require("reshape")
require("ggplot2")
require("leaflet")
require("leaflet.esri")
print ("Verificando Licencia de arcgis pro")
arc.check_product()

## Datos de entrada y Salida

In [ ]:
querybase <- '80849753'
salida_shape <-"C:\\Esri\\env\\resultados.gdb\\Baez"
salida_reporte <- "C:\\Esri\\env\\reporte_baez.html"

## Constantes Usadas en el Proceso

In [ ]:
outputformathtml = "html_document"
#outputformatpdf = "pdf_document"
output_report_format = outputformathtml
#pathDepartamentos <-"C:\\esri\\r-arcgis-tools\\datasets\\colombia.gdb\\deptos"
pathDepartamentos <- "https://services1.arcgis.com/I4YYbPSw13ugmbAP/ArcGIS/rest/services/GDB_ECV_2da_parte_gdb/FeatureServer/0" # "C:\\esri\\r-arcgis-tools\\datasets\\colombia.gdb\\deptos"
pathReporte <- "c:\\esri\\r-arcgis-tools\\src\\ReportSecop.Rmd"
portal_socrata = "https://www.datos.gov.co"
resource ="/resource/c6dm-udt9.json?"
columnas_interes <- list(c("identificacion_del_contratista",
                             "nom_raz_social_contratista",
                             "plazo_de_ejec_del_contrato",
                             "tiempo_adiciones_en_meses",
                             "anno_firma_del_contrato",
                             "dpto_y_muni_contratista",
                             "nombre_de_la_entidad",
                             "valor_contrato_con_adiciones",
                             "numero_de_proceso",
                             "numero_del_contrato",
                             "ruta_proceso_en_secop_i"))
orderbycolumns <- list(c("anno_firma_del_contrato","valor_contrato_con_adiciones"))


In [ ]:
columnas <- stri_join_list(columnas_interes,sep=",")
orderby <- stri_join_list(orderbycolumns,sep=",")
columnas_rename <- list(c("ID",
                            "CONTRATISTA",
                            "PLAZO",
                            "ADICIONES",
                            "ANO",
                            "DEPARTAMENTO",
                            "ENTIDAD",
                            "VALOR",
                            "PROCESO",
                            "CONTRATO",
                            "URL"))

# Proceso de consulta de capa remota de ArcGIS Server

In [ ]:
arcMuni <- arc.open(path=pathDepartamentos)
d_sub <- arc.select(arcMuni, fields = "*", where_clause = "1=1", sr = 4326)
element<- arc.data2sp(d_sub)

In [ ]:
mapaentrada <- leaflet(element) %>%
                addProviderTiles(providers$Esri)  %>%
                addPolygons()


In [ ]:
mapaentrada
element@data

In [ ]:
element@data$NOMBRE <- toupper(element@data$DPTO_CNMBR)
element@data$NOMBRE <-gsub("Á","A",element@data$NOMBRE)
element@data$NOMBRE <-gsub("É","E",element@data$NOMBRE)
element@data$NOMBRE <-gsub("Í","I",element@data$NOMBRE)
element@data$NOMBRE <-gsub("Ó","O",element@data$NOMBRE)
element@data$NOMBRE <-gsub("Ú","U",element@data$NOMBRE)
element@data$NOMBRE <-gsub("\\.","",element@data$NOMBRE)
element@data$NOMBRE <-gsub(",","",element@data$NOMBRE)
element@data

### Consulta al portal de Secop

Para hacer consultas por el API de SODA ver [referencia](https://dev.socrata.com/docs/queries/)

In [ ]:
query<- paste("$select=",columnas,sep="")
query <- paste(query,"&$q=",querybase,sep="")
query<- paste(query,"&$order=",orderby,sep="")

In [ ]:
secop_portal_url <- paste(portal_socrata,resource,sep="")
request <- paste(secop_portal_url,query,sep="")
df <- read.socrata(request)
df

### Calculo de Agregaciones, por año y graficas exploratorias

In [ ]:
df_cp <- setNames(df, columnas_rename[[1]]) %>% mutate(VALOR = as.numeric(VALOR)/1000000)
summary_year <- group_by(df_cp, ANO,DEPARTAMENTO) %>% 
                    summarize(TOTAL_CONT = sum(VALOR, na.rm=TRUE),
                    PROM = mean(VALOR, na.rm = TRUE),
                    MAX = max(VALOR, na.rm = TRUE),
                    MIN = min(VALOR, na.rm = TRUE))
  
summary_tot <- select (summary_year,ANO:TOTAL_CONT)
ano_ciudad <- cast(summary_tot , ANO~DEPARTAMENTO,value="TOTAL_CONT")
ano_ciudad[is.na(ano_ciudad)] <- 0
grafica_mun <- ggplot(summary_tot, aes(x=DEPARTAMENTO, y=TOTAL_CONT )) + geom_col(aes(fill = ANO), width = 0.7) + coord_flip() 
grafica_mun
summary_year

In [ ]:
lista_graficas = c()
for (i in 2:ncol(ano_ciudad))
{
    subset <- ano_ciudad[,c(1,i)]
    depto <- colnames(subset)[2]
    label1 <- paste("Contratado en el depto de ",depto,sep="")
    colnames(subset)<-c('ano','valor')
    grafica_depto <- ggplot(subset, aes(x=ano, y = valor,  group=1)) + geom_point()  + geom_line()  + labs(title=label1)+ theme_minimal()
    lista_graficas <- c(lista_graficas,list(grafica_depto))
}

In [ ]:
lista_graficas[1]

## Acomodando tablas de datos para unir tablas por columna DEPARTAMENTO

In [ ]:
departamento_ano <- cast(summary_tot , DEPARTAMENTO~ANO,value="TOTAL_CONT")
departamento_ano[is.na(departamento_ano)] <- 0
departamento_ano$DEPARTAMENTO <- toupper(departamento_ano$DEPARTAMENTO)
departamento_ano$DEPARTAMENTO <-gsub("Á","A",departamento_ano$DEPARTAMENTO)
departamento_ano$DEPARTAMENTO <-gsub("É","E",departamento_ano$DEPARTAMENTO)
departamento_ano$DEPARTAMENTO <-gsub("Í","I",departamento_ano$DEPARTAMENTO)
departamento_ano$DEPARTAMENTO <-gsub("Ó","O",departamento_ano$DEPARTAMENTO)
departamento_ano$DEPARTAMENTO <-gsub("Ú","U",departamento_ano$DEPARTAMENTO)
departamento_ano$DEPARTAMENTO <-gsub(",","",departamento_ano$DEPARTAMENTO)
departamento_ano$DEPARTAMENTO <-gsub("\\.","",departamento_ano$DEPARTAMENTO)

In [ ]:
departamento_ano<-departamento_ano %>% mutate (PROMEDIO=Reduce("+",.[2:ncol(departamento_ano)])/(ncol(departamento_ano)-1))
departamento_ano<-departamento_ano %>% mutate (TOTAL=Reduce("+",.[2:ncol(departamento_ano)]))
departamento_ano

  

In [ ]:
  promedio_por_departamento <- departamento_ano %>% select(DEPARTAMENTO,PROMEDIO)

## Join de datos espaciales con datos alfanumericos para producir cartografia de salida

In [ ]:
mp<- element@data
OutDF<- sqldf("select a.*, b.* from mp as a left join departamento_ano as b on a.NOMBRE = b.DEPARTAMENTO",drv="SQLite" )
OutDF[is.na(OutDF)] <- 0
element2 <- SpatialPolygonsDataFrame(element,OutDF,match.ID = FALSE)
element2 <- subset(element2, TOTAL>0)  
element2@data

In [ ]:
if(nrow(element2@data)>1)
  {
    renderer <- colorQuantile("RdYlBu",domain=element2$PROMEDIO, reverse=T)
    mapa_salida <- leaflet(element2) %>%
      addProviderTiles(providers$Esri)  %>%
      addPolygons(
        color=~renderer(PROMEDIO)
      ) %>%
      addLegend(pal=renderer, values = element2$PROMEDIO,title="Promedio contratado")
    
  } else {
      mapa_salida <- leaflet(element2) %>%
      addProviderTiles(providers$Esri) %>%
        addPolygons()
}


In [ ]:
mapa_salida

In [ ]:
arc.write(salida_shape,element2)

## Construcción de Reporte de Salida

In [ ]:
parametros_reporte <- list(
    tabla_consulta = df_cp,
    texto_consulta = querybase, 
    tabla_resumen = summary_year,
    mapa_salida = mapa_salida,
    grafico_municipios = grafica_mun,
    graficas_individuales = lista_graficas,
    promedio_por_departamento = promedio_por_departamento
  )
  render(pathReporte, output_file = salida_reporte,
         params=parametros_reporte,
         output_format = output_report_format,encoding = "UTF-8" )